# Lab 1 - drzewa decyzyjne
*"pro" version*

Autorstwa:
- Wojciech Kot  151879
- Julia Samp    151775
z grupy L7, zajęcia piątkowe 11:45

Step - by - step - Wersja podstawowa (pro znajduje się poniżej)

Wpierw, wgranie danych do dataframe'u zapewnianego przez pandas
(ze względu na wygodę, jaką zapewnia oraz obsługę .csv)

In [139]:
import pandas


def prepare_pandas_df_for_basic_problem(path):
    data = pandas.read_csv(path)
    data = data.drop(['PassengerId', 'Name'], axis=1)
    data['Age'] = pandas.cut(data['Age'], bins=[0, 20, 40, 100], labels=['young', 'middle', 'old'], right=True)
    return data


dane2 = prepare_pandas_df_for_basic_problem("titanic-homework.csv")
print(dane2)

    Pclass     Sex     Age  SibSp  Parch  Survived
0        3    male  middle      1      0         0
1        1  female  middle      1      0         1
2        3  female  middle      0      0         1
3        1  female  middle      1      0         1
4        3    male  middle      0      0         0
..     ...     ...     ...    ...    ...       ...
95       3    male     old      0      0         0
96       1    male     old      0      0         0
97       1    male  middle      0      1         1
98       2  female  middle      0      1         1
99       2    male  middle      1      0         0

[100 rows x 6 columns]


Kiedy mamy już poprawnie załadowane dane, należy zacząć budować drzewo, jednak najpierw kilka funkcji, które mogą nam się przydać przy obsłudze dataframe'u:

In [140]:
# Aby wiedzieć ILE i jakie rozgałęzienia będzie miało nasze drzewo, gdy podzielimy na tym atrybucie
def possible_values(df, column_name):
    # zwraca posortowaną listę unikalnych wartości z danej kolumny
    return sorted(df[column_name].unique().tolist())


# Filtr odsiewający kolumnę Survived
def get_column_names(df):
    # zwraca listę kolumn, z wyłączeniem kolumny "z decyzją"
    return [col for col in df.columns if col != "Survived"]


# Funkcja zliczająca ile osób przeżyło i ile nie przeżyło, dla danej wartości w danej kolumnie, używana w split()
def filter_count(df, column_name, value):
    filtered_df = df[df[column_name] == value]
    p = int(filtered_df['Survived'].sum())
    ile_wszystkich = len(filtered_df)
    # zwraca parę [przeżyło, nie-przeżyło]
    return [p, ile_wszystkich - p]


# Funkcja przygotowująca dane w przypadku podziału na danym atrybucie (danej kolumnie)
def split(df, column_name):
    l = []
    for val in possible_values(df, column_name):
        l.append(filter_count(df, column_name, val))
    # zwraca listę par [przeżyli, nie-przeżyli], dla każdego atrybutu
    return l


A teraz funkcje matematyczne z zajęć, które będą nam potrzebne przy podejmowaniu decyzji podczas budowania drzewa

In [141]:
import math

Podstawowa funkcja licząca entropię. Danymi wejściowymi jest lista list, gdzie listy wewnętrzne mają tylko 2 elementy, np. [[0, 2], [1, 1], [4, 2]].
def entropy(dane):
    e = 0
    suma = sum(dane)
    for i in dane:
        if i != 0:
            e -= (i / suma) * math.log(i / suma, 2)
    return e


# funkcja obliczająca entropię warunkową z wykorzystaniem zdefiniowanej wcześniej funkcji entropii. Dane wejściowe są identyczne jak w funkcji wyżej.
def conditional_entropy(dane):
    ce = 0
    ile = 0
    for i in dane:
        ile += sum(i)
    for i in dane:
        e = entropy(i)
        ce += e * sum(i) / ile
    return ce


# funkcja obliczająca zysk informacyjny przy użyciu dwóch poprzednich funkcji. Forma danych wejściowych jest taka sama jak w poprzednich funkcjach.
def information_gain(dane):
    y = 0
    n = 0
    for i in dane:
        y += i[0]
        n += i[1]
    ig = entropy([y, n]) - conditional_entropy(dane)

    return ig


# funkcja obliczająca gain ratio przy użyciu intrinsic info. Dane wejściowe mają identyczną formę jak w poprzednich funkcjach.
def gain_ratio(dane):
    ig = information_gain(dane)
    ii = 0
    ile = 0
    for i in dane:
        ile += sum(i)
    for i in dane:
        ii -= (sum(i) / ile) * math.log(sum(i) / ile, 2)
    gr = ig/ii
    return gr

Oraz wreszcie główna funkcja służąca do budowy drzewa decyzyjnego:

In [142]:
def build_tree_with_decision(df, tree, columns, parent=None, parent_id="root"):
    # Jeżeli mamy pewną decyzję, to dodajemy liść z tą decyzją
    if len(df['Survived'].unique()) == 1:
        count_survived = len(df[df['Survived'] == 1])
        count_dead = len(df[df['Survived'] == 0])
        if df['Survived'].values[0] == 1:
            tree.create_node(f"Survived: {count_survived}", parent_id, parent=parent)
        else:
            tree.create_node(f"Dead: {count_dead}", parent_id, parent=parent)
        return

    # Liczymy info-gain dla każdego możliwego w tym momencie podziału
    best_gain = 0.0
    best_column = None
    for column in columns:
        ig = information_gain(split(df, column))
        if ig > best_gain:
            best_gain = ig
            best_column = column

    # Dodatkowy warunek stopu rekurencji (nigdy nie powinien zajść przy braku sprzecznych danych, ale chroni przed błędami)
    if best_gain == 0 or len(columns) == 0:
        count_survived = len(df[df['Survived'] == 1])
        count_dead = len(df[df['Survived'] == 0])
        # Tworzy liść z wynikiem
        tree.create_node(f"Niejednoznaczny wynik (Survived: {count_survived}, Dead: {count_dead})", parent_id, parent=parent)
        return

    # kolumnę na podstawię której właśnie dzielimy, usuwamy, przekazując do węzła-dziecka
    remaining_columns = columns.copy()
    remaining_columns.remove(best_column)
    # Nadajemy unikalne node_id, składające się z nazwy atrybutu, na podstawie którego dzielimy (best_column), oraz id węzła-rodzica, tak aby zapewnić unikalność w całym drzewie
    node_id = f"{best_column}_{parent_id}"

    # Zliczamy parametry w danym węźle i dodajemy go do drzewa
    count_survived = len(df[df['Survived'] == 1])
    count_dead = len(df[df['Survived'] == 0])

    # Tutaj dodajemy do drzewa właśnie przetwarzany węzeł
    param = parent_id.split('_')[-1]
    tree.create_node(f"{param} - {best_column} (Survived: {count_survived}, Dead: {count_dead})", node_id, parent=parent)

    # Rekurencyjnie budujemy dalej całe drzewo, przekazując do węzła-dziecka parametry pomniejszone o zużyty już parametr, oraz odpowiednio odfiltrowane dane (df i kolumn) oraz swoje id, które użyje do nadania sobie id.
    for val in possible_values(df, best_column):
        filtered_df = df[df[best_column] == val]
        id_for_child = f"{node_id}_{val}"
        build_tree_with_decision(filtered_df, tree, remaining_columns, node_id, id_for_child)

Ostatecznie, driver code, czyli wywołanie przygotowanych powyżej funkcji:

In [143]:
from treelib import Tree

dane = prepare_pandas_df_for_basic_problem("titanic-homework.csv")
decision_tree = Tree()
columns = get_column_names(dane)
build_tree_with_decision(dane, decision_tree, columns)
print(decision_tree.show(stdout=False))

root - Sex (Survived: 40, Dead: 60)
├── female - SibSp (Survived: 33, Dead: 7)
│   ├── 1 - Pclass (Survived: 11, Dead: 4)
│   │   ├── 3 - Parch (Survived: 2, Dead: 4)
│   │   │   ├── Dead: 4
│   │   │   ├── Survived: 1
│   │   │   └── Survived: 1
│   │   ├── Survived: 4
│   │   └── Survived: 5
│   ├── 3 - Age (Survived: 2, Dead: 1)
│   │   ├── Dead: 1
│   │   └── Survived: 2
│   ├── Dead: 1
│   ├── Dead: 1
│   ├── Survived: 1
│   └── Survived: 19
└── male - Pclass (Survived: 7, Dead: 53)
    ├── 1 - Age (Survived: 4, Dead: 9)
    │   ├── Dead: 7
    │   ├── middle - SibSp (Survived: 3, Dead: 1)
    │   │   ├── Dead: 1
    │   │   └── Survived: 3
    │   └── young - SibSp (Survived: 1, Dead: 1)
    │       ├── Dead: 1
    │       └── Survived: 1
    ├── 2 - Age (Survived: 2, Dead: 7)
    │   ├── Dead: 1
    │   ├── Dead: 6
    │   └── Survived: 2
    └── 3 - Parch (Survived: 1, Dead: 37)
        ├── 1 - Age (Survived: 1, Dead: 3)
        │   ├── Dead: 3
        │   └── Survived: 1
     

# Wersja Pro

Większość funkcji pozostaje taka sama, jednak potrzebna jest dodatkowa funkcja, która wyznaczy granice decyzyjne na atrybucie "Age"

In [144]:
from itertools import combinations


# Wgranie danych do dataframe'u, tym razem bez mapowania Age na sztucznie nałożone granice, ale z sortowaniem po "Age" oraz "Survived"
def prepare_pandas_df(path):
    data = pandas.read_csv(path)
    data = data.drop(['PassengerId', 'Name'], axis=1)
    data = data.sort_values(by=['Age', 'Survived']).reset_index(drop=True)
    return data


def map_splits_on_data(df, splits):
    # Funkcja pomocnicza do przypisania klasy na podstawie progów podziału
    def assign_class(age):
        for i, split in enumerate(splits):
            if age <= split:
                return i
        return len(splits)  # Jeśli większy niż ostatni próg, przypisz ostatnią klasę

    # Zastępujemy wartości w kolumnie "Age" przypisanymi klasami (ich numerami)
    df['Age'] = df['Age'].apply(assign_class)
    return df


# Funkcja wyliczająca najlepsze progi podziału dla ciągłej zmiennej "Age"
def find_best_splits_for_continuous(df, num_classes):
    if num_classes <= 1:
        return []
    if num_classes >= 4:
        print(f'To chwilę potrwa, nawet na naszym "małym" datasecie num_classes = {num_classes}')

    # Określenie potencjalnych progów podziału (średnie wartości między punktami gdzie zmienia się klasa)
    potential_splits = []
    for i in range(len(df) - 1):
        if df['Survived'][i] != df['Survived'][i + 1]:
            potential_split = (df['Age'][i] + df['Age'][i + 1]) / 2
            potential_splits.append(potential_split)

    # Przygotowanie do znalezienia najlepszych progów podziału
    best_splits = []
    best_gain = -1

    # Kombinacje potencjalnych progów (do sprawdzenia brute-force najlepszej kombinacji)
    possible_splits = list(combinations(potential_splits, num_classes - 1))
    for split_combination in possible_splits:
        splits = []
        prev_split = 0.0

        # Podział na grupy zgodnie z kombinacją progów
        for split in split_combination:
            current_split = df[(df['Age'] > prev_split) & (df['Age'] <= split)]
            counts = [len(current_split[current_split['Survived'] == cls]) for cls in df['Survived'].unique()]
            splits.append(counts)
            prev_split = split

        # Dodanie ostatniego przedziału
        last_split = df[df['Age'] > split_combination[-1]]
        counts = [len(last_split[last_split['Survived'] == cls]) for cls in df['Survived'].unique()]
        splits.append(counts)

        # Znalezienie najlepszej kombinacji progów
        ig = information_gain(splits)
        if ig > best_gain:
            best_gain = ig
            best_splits = split_combination

    return sorted(best_splits)


Driver code dla przypadku pro

In [145]:
dane = prepare_pandas_df("titanic-homework.csv")
best_splits = find_best_splits_for_continuous(dane, 3)
# best_splits = (20, 40)   # Przypadek z wersji na 4.0
dane = map_splits_on_data(dane, best_splits)

decision_tree = Tree()
columns = get_column_names(dane)
build_tree_with_decision(dane, decision_tree, columns)
print(decision_tree.show(stdout=False))

root - Sex (Survived: 40, Dead: 60)
├── female - SibSp (Survived: 33, Dead: 7)
│   ├── 1 - Pclass (Survived: 11, Dead: 4)
│   │   ├── 3 - Parch (Survived: 2, Dead: 4)
│   │   │   ├── Dead: 4
│   │   │   ├── Survived: 1
│   │   │   └── Survived: 1
│   │   ├── Survived: 4
│   │   └── Survived: 5
│   ├── 3 - Age (Survived: 2, Dead: 1)
│   │   ├── Dead: 1
│   │   └── Survived: 2
│   ├── Dead: 1
│   ├── Dead: 1
│   ├── Survived: 1
│   └── Survived: 19
└── male - Pclass (Survived: 7, Dead: 53)
    ├── 1 - SibSp (Survived: 4, Dead: 9)
    │   ├── 0 - Age (Survived: 4, Dead: 4)
    │   │   ├── 2 - Parch (Survived: 3, Dead: 4)
    │   │   │   ├── Niejednoznaczny wynik (Survived: 1, Dead: 1)
    │   │   │   └── Niejednoznaczny wynik (Survived: 2, Dead: 3)
    │   │   └── Survived: 1
    │   ├── Dead: 1
    │   └── Dead: 4
    ├── 2 - Age (Survived: 2, Dead: 7)
    │   ├── Dead: 1
    │   ├── Dead: 6
    │   └── Survived: 2
    └── 3 - Parch (Survived: 1, Dead: 37)
        ├── 1 - Age (Survived: 

W wariancie na 5.0, a więc z progami wybieranymi dynamicznie pod względem information gain można zauważyć, że pojawiają się w drzewie liście z niesprecyzowanymi danymi. Najpewniej jest to efekt dobrania danych w taki sposób, aby wariant na 4.0 takich nie zawierał, ale mogę się mylić.